In [1]:
import datetime
import time
import pandas as pd
from pandas import json_normalize
import json
import cufflinks as cf
cf.go_offline()
import plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from requests_html import HTMLSession
session = HTMLSession()

import FinanceDataReader as fdr
# https://github.com/FinanceData/FinanceDataReader

ITEMS = pd.read_excel('items.xlsx').fillna('').set_index('symbol').T.to_dict()

def quote_name(symbol):
    if ITEMS[symbol]['exchange']:
        return ITEMS[symbol]['exchange'] + ':' + symbol 
    return symbol

def code_title(symbol):
    return ITEMS[symbol]['title']

def str_to_timestamp(date):
    obj_datetime = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return datetime_to_timestamp(obj_datetime)

def datetime_to_timestamp(dt):
    return int(time.mktime(dt.timetuple()))

def transform_ohlc(data):
    result = {}
    if 't' in data.keys():
        result['Date'] = data.pop('t')
        for i, v in enumerate(result['Date']):
            result['Date'][i] = datetime.datetime.fromtimestamp(v)
    if 'o' in data.keys():
        result['Open'] = data.pop('o')
    if 'h' in data.keys():
        result['High'] = data.pop('h')
    if 'l' in data.keys():
        result['Low'] = data.pop('l')
    if 'c' in data.keys():
        result['Close'] = data.pop('c')
    if 'v' in data.keys():
        result['Volume'] = data.pop('v')
        
    df = pd.DataFrame.from_dict(result)
    return df.set_index('Date')


#https://tvc4.investing.com/7a400a377dbd042ec620a468748b4bd6/1641706107/1/1/8/search?limit=30&query=NASDAQ&type=&exchange=
# https://tvc4.investing.com/8a1d14438d13e084d0c5646d448be5d0/1641706529/18/18/88/search?limit=30&query=%EC%82%BC%EC%84%B1&type=&exchange=
host_kor = 'https://tvc4.investing.com/8a1d14438d13e084d0c5646d448be5d0/1641706529/18/18/88/'
host_eng = 'https://tvc4.investing.com/2a14e553c0cc99adc41f11079f65b4cd/1653822410/1/1/8/'
host = host_kor


def history(code, resolution, from_date, to_date=None):
    from_timestamp = str_to_timestamp(from_date + ' 00:00:00' if len(from_date) == 10 else from_date)
    if not to_date:
        to_timestamp = datetime_to_timestamp(datetime.datetime.now())
    else:     
        to_timestamp = str_to_timestamp(to_date + ' 23:59:59' if len(to_date) == 10 else to_date)
 
    data = history_raw(code, resolution, from_timestamp, to_timestamp)
    if len(data['t']) >= 5000:
        while True:
            data2 = history_raw(code, resolution, data['t'][-1] + 1, to_timestamp)
            data['t'] = data['t'] + data2['t']
            data['o'] = data['o'] + data2['o']
            data['h'] = data['h'] + data2['h']
            data['l'] = data['l'] + data2['l']
            data['c'] = data['c'] + data2['c']
            data['v'] = data['v'] + data2['v']
            if len(data2['t']) < 5000: break
                
    return transform_ohlc(data)

def history_raw(code, resolution, from_timestamp, to_timestamp):
    uri = f'history?symbol={code}&resolution={resolution}&from={from_timestamp}&to={to_timestamp}'
#     print(host+uri)
    r = session.get(host + uri)
    return json.loads(r.html.text)    

def quotes(symbols):
    symbols_text = ','.join(map(quote_name, symbols))
    uri = f'quotes?symbols={symbols_text}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)
    d = [row['v'] for row in data['d']]
    return pd.DataFrame(d)

def symbols(symbol):
    uri = f'symbols?symbol={symbol}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)    
    return json_normalize(data)

def search(query, limit=50):
    uri = f'search?limit={limit}&query={query}&type=&exchange='
    r = session.get(host + uri)
    data = json.loads(r.html.text)    
    return pd.DataFrame(data)

def candle(symbol, resolution, from_date, to_date=None):
    item = ITEMS[symbol]
    return candle_df(history(item['code'], resolution, from_date, to_date), item['digit'])

def candle_df(df, digit=0):
    df['Open'] = round(df['Open'], digit)
    df['High'] = round(df['High'], digit)
    df['Low'] = round(df['Low'], digit)
    df['Close'] = round(df['Close'], digit)
    df['Pre_Close'] = df['Close'].shift()
    df['Gap'] = df['Open']-df['Pre_Close']
    df['Change'] = df['Close']-df['Pre_Close']
    df['Change_Rate'] = round((df['Close']/df['Pre_Close']-1)*100, 2)
    df['Body'] = df['Close']-df['Open']
    df['Height'] = df['High']-df['Low']
    df['Height_Rate'] = round((df['High']/df['Low']-1)*100, 2)
    df['Head'] = df['High'] - [row['Close'] if row['Body'] > 0 else row['Open'] for i, row in df.iterrows()]
    df['Tail'] = [row['Close'] if row['Body'] < 0 else row['Open'] for i, row in df.iterrows()] - df['Low']
    df['RHead'] = round((df['Head'] / df['Height'])*100, 2)
    df['RBody'] = round((abs(df['Body']) / df['Height'])*100, 2)
    df['RTail'] = round((df['Tail'] / df['Height'])*100, 2)
    return df
    
def candlestick(df):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.1, 0.7])
    fig.add_trace(go.Candlestick(x=df.index.strftime("%y.%m.%d %H:%M"),
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'], showlegend=False),
               row=1, col=1)
    fig.add_trace(go.Bar(x=df.index.strftime("%y.%m.%d %H:%M"), y=df['Volume'], showlegend=False),
               row=3, col=1)
#     fig.update(layout_xaxis_rangeslider_visible=False)
    fig.update_layout(height=700)
    fig.show()
    
def ohlc(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index.strftime("%y.%m.%d %H:%M"),
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'])])
#     fig.update_xaxes(
#         rangeslider_visible=True,
#         rangebreaks=[
#             # NOTE: Below values are bound (not single values), ie. hide x to y
#             dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
# #             dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
#             # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
#         ]
#     )    
    fig.update_layout(
#         autosize=False,
#         width=500,
        height=700
#         margin=dict(
#             l=50,
#             r=50,
#             b=100,
#             t=100,
#             pad=4
#         ),
#         paper_bgcolor="LightSteelBlue",
    )

    fig.show()    
    
def df_compare(targets, resolution, start_day, end_day=None):
    df_list = [ history(target, resolution, start_day, end_day)['Close'] for target in targets ]
    df = pd.concat(df_list, axis=1)
    df.columns = [ ITEMS[target]['title'] for target in targets ] 
    return df

def index_chart(df):
    df_plot = df / df.iloc[0] - 1.0
    df_plot['t'] = df_plot.index.strftime("%y.%m.%d %H:%M")
    df_plot = df_plot.set_index('t')
    df_plot.iplot() 
    
def one_symbol_plot(symbol, date, count=2, resolution='5'):
    df = one_symbol(symbol, date, count, resolution)
    return df.iplot(title=ITEMS[symbol]['title'])
    

def one_symbol(symbol, date, count=2, resolution='5'):
    item = ITEMS[symbol]
    start = datetime.datetime.strptime(date + ' 07:00:00', '%Y-%m-%d %H:%M:%S')
    end = start + datetime.timedelta(days=1) - datetime.timedelta(seconds=1)
    df = one_symbol_adj(candle_df(history(item['code'], resolution, start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')), item['digit']))
    
    while count > 1:
        start = start - datetime.timedelta(days=1)
        end = end - datetime.timedelta(days=1)
        try:
            idf = one_symbol_adj(candle_df(history(item['code'], resolution, start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')), item['digit']))
            df = pd.merge(df, idf, on='Date', how='right')
            count = count - 1
        except:
            pass
    return df

def one_symbol_adj(df):
    df = df[['Close']]
    df.rename(columns = {'Close': df.index[0].strftime('%Y-%m-%d')}, inplace = True)
    df.index = df.index.strftime('%H:%M')
    return df

def price_symbol_plot(symbol, date1, date2=None, resolution='5'):
    df = price_symbol(symbol, date1, date2, resolution)
    fig = df.iplot(title=f'{date1}~{date2}', asFigure=True)
    fig.update_layout(
        height=700,
        xaxis=dict(type="category")
    )
    fig.show()

def price_symbol(symbol, date1, date2=None, resolution='5'):
    item = ITEMS[symbol]
    start = datetime.datetime.strptime(date1 + ' 07:00:00', '%Y-%m-%d %H:%M:%S')
    if (date2):
        end = datetime.datetime.strptime(date2 + ' 06:59:59', '%Y-%m-%d %H:%M:%S')
    else: 
        end = start + datetime.timedelta(days=1) - datetime.timedelta(seconds=1)
    df = price_symbol_adj(candle_df(history(item['code'], resolution, start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')), item['digit']), item['title'])

    return df

def price_symbol_adj(df, title):
    df = df[['Close']]
    df.rename(columns = {'Close': df.index[0].strftime(title)}, inplace = True)
    # df.index = df.index.strftime('%Y-%m-%d %H:%M')
    return df

def index_symbols_plot(symbols, date1, date2=None, resolution='5'):
    df = index_symbols(symbols, date1, date2, resolution)
    fig = df.iplot(title=f'{date1}~{date2}', asFigure=True)
    # fig = df.iplot(title=f'{date1}~{date2}', asFigure=True, mode="lines+markers", size=4.0)
    # fig.layout = dict(xaxis=dict(type="category"))
    fig.update_layout(
        height=700,
        xaxis=dict(type="category")
    )
    # fig.update_xaxes(tickformat = '%H:%M')
    # fig.update_xaxes(
    #     rangebreaks=[
    #         dict(values=break_times(df))
    #         # NOTE: Below values are bound (not single values), ie. hide x to y
    #         # dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
    #         # dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
    #         # dict(values=["2020-12-25", "2021-01-01"])  # hide holidays (Christmas and New Year's, etc)
    #     ]
    # )
    fig.show()


def index_symbols(symbols, date1, date2=None, resolution='5'):
    item = ITEMS[symbols[0]]
    start = datetime.datetime.strptime(date1 + ' 07:00:00', '%Y-%m-%d %H:%M:%S')
    if (date2):
        end = datetime.datetime.strptime(date2 + ' 06:59:59', '%Y-%m-%d %H:%M:%S')
    else: 
        end = start + datetime.timedelta(days=1) - datetime.timedelta(seconds=1)
    df = index_symbol_adj(candle_df(history(item['code'], resolution, start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')), item['digit']), item['title'], resolution)
    
    for i in range(1, len(symbols)):
        item = ITEMS[symbols[i]]
        idf = index_symbol_adj(candle_df(history(item['code'], resolution, start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')), item['digit']), item['title'], resolution)
        df = pd.merge(df, idf, on='Date', how='left')
        
    return df

def index_symbol_adj(df, title, resolution):
    df = df[['Close']]
    df['Close'] = df['Close'] / df.iloc[0]['Close'] - 1.0
    df.rename(columns = {'Close': df.index[0].strftime(title)}, inplace = True)
    # df.index = df.index.strftime('%Y-%m-%d %H:%M')
    return df

def break_times(df):
    delta = df.index[1] - df.index[0]
    cindex = df.index[0]
    breaks = []
    for i, index in enumerate(df.index):
        while cindex < index:
            breaks.append(cindex)
            cindex = cindex + delta
        cindex = cindex + delta
    return breaks


def candle_chart(symbol, resolution='60', date1=None, date2=None):
    item = ITEMS[symbol]
    if not date1:
        date1 = datetime.datetime.now().strftime('%Y-%m-%d') + ' 07:00:00'
    if not date2:
        date2 = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    df = candle(symbol, resolution, date1, date2)
    qf = cf.QuantFig(df, title=f"{item['title']}({date1}~{date2},{resolution})")
    # qf.add_bollinger_bands()
    # qf.add_volume()
    qf.add_macd()
    qf.add_rsi()
    # qf.add_sma([10,20],width=2,color=['green','lightgreen'],legendgroup=True)
    # qf.add_rsi(periods=20,color='java')
    fig = qf.figure()
    fig.update_layout(
        height=700,
        xaxis=dict(type="category", showticklabels=False)
    )
    fig.show()   
    return df 
    

In [43]:
search('나스닥')

,symbol,full_name,description,type,ticker,exchange
0,NQc1,CME:NQc1,나스닥 100 선물,선물 지수,8874,CME
1,Q580015,서울:Q580015,KB 레버리지 나스닥 100 ETN,상장지수펀드,1176871,서울
2,Q570044,서울:Q570044,TRUE 인버스 2X 나스닥 100 ETN,상장지수펀드,1145864,서울
3,203780,서울:203780,미래에셋 TIGER 나스닥바이오증권상장지수투자신탁(주식),상장지수펀드,953469,서울
4,133690,서울:133690,TIGER 미국나스닥100,상장지수펀드,953468,서울
5,IXIC,나스닥:IXIC,나스닥종합지수,지수,14958,나스닥
6,NDX,나스닥:NDX,나스닥 100,지수,20,나스닥
7,NDAQ,나스닥:NDAQ,나스닥,주식,6395,나스닥
8,USTECH,US Indexes:USTECH,나스닥 100 선물,지수,1175151,US Indexes
9,NDAQ-RM,MCX:NDAQ-RM,나스닥,주식,1180696,MCX


In [94]:
quotes(['SI','CL','NG','HG','GC','VX','NQc1','US500','USD/KRW','005930','035420'])

,ch,chp,short_name,exchange,description,lp,ask,bid,spread,open_price,high_price,low_price,prev_close_price,volume
0,-0.335,-1.50,은,,은 선물,21.940,21.965,21.955,0,22.378,22.525,21.860,22.275,"35,888"
1,3.39,2.90,WTI유,,WTI유 선물,120.26,120.37,120.34,0,117.60,120.46,115.23,116.87,"225,138"
2,0.054,0.64,천연가스,,천연가스 선물,8.539,8.541,8.533,0,8.415,8.693,8.267,8.485,"71,100"
3,-0.0753,-1.65,구리,,구리 선물,4.4772,4.4805,4.4740,0,4.5555,4.5768,4.4585,4.5525,0
4,-17.50,-0.94,금,,금 선물,1853.90,"1,854.30","1,854.10",0,"1,872.90","1,878.60","1,849.70",1871.40,"110,603"
5,0.43,1.68,S&P 500 VIX,,S&P 500 VIX 선물,26.03,25.98,25.88,0,25.50,26.70,25.43,25.60,"59,826"
6,-342.75,-2.66,나스닥 100,CME,나스닥 100 선물,12551.00,"12,550.25","12,544.75",0,"12,896.00","12,945.25","12,503.00",12893.75,0
7,-66.7,-1.60,S&P 500,US Indexes,S&P 500 선물,4110.1,"4,110.4","4,109.8",0,"4,178.5","4,189.8","4,098.4",4176.8,0
8,10.65,0.86,USD/KRW,실시간 FX,USD/KRW,1251.83,"1,252.16","1,251.50",66,"1,241.18","1,252.25","1,238.47",1241.18,"2,662"
9,100,0.15,삼성전자,서울,삼성전자,66800,"66,900","66,800",0,"67,200","67,300","66,800",66700,"8,171,026"


In [116]:
quotes(ITEMS.keys())

,ch,chp,short_name,exchange,description,lp,ask,bid,spread,open_price,high_price,low_price,prev_close_price,volume
0,-17.50,-0.94,금,,금 선물,1853.90,"1,854.30","1,854.10",0,"1,872.90","1,878.60","1,849.70",1871.40,"110,603"
1,-0.335,-1.50,은,,은 선물,21.940,21.965,21.955,0,22.378,22.525,21.860,22.275,"35,888"
2,-0.0753,-1.65,구리,,구리 선물,4.4772,4.4805,4.4740,0,4.5555,4.5768,4.4585,4.5525,0
3,3.39,2.90,WTI유,,WTI유 선물,120.26,120.37,120.34,0,117.60,120.46,115.23,116.87,"225,138"
4,0.054,0.64,천연가스,,천연가스 선물,8.539,8.541,8.533,0,8.415,8.693,8.267,8.485,"71,100"
5,-324.6,-0.98,Dow Jones,US Indexes,다우존스 30 선물,32923.7,"32,926.5","32,920.9",0,"33,247.6","33,321.6","32,838.7",33248.3,0
6,-66.7,-1.60,S&P 500,US Indexes,S&P 500 선물,4110.1,"4,110.4","4,109.8",0,"4,178.5","4,189.8","4,098.4",4176.8,0
7,-342.75,-2.66,나스닥 100,CME,나스닥 100 선물,12551.00,"12,550.25","12,544.75",0,"12,896.00","12,945.25","12,503.00",12893.75,0
8,0.43,1.68,S&P 500 VIX,,S&P 500 VIX 선물,26.03,25.98,25.88,0,25.50,26.70,25.43,25.60,"59,826"
9,0.353,0.35,미국 달러 지수,,미국 달러 지수 선물,102.185,102.205,102.175,0,101.762,102.245,101.660,101.832,"14,929"


In [3]:
df = candle_chart('NQc1', '60', '2022-05-20 07:00:00')

In [17]:
df.tail(10)

,Open,High,Low,Close,Volume,Pre_Close,Gap,Change,Change_Rate,Body,Height,Height_Rate,Head,Tail,RHead,RBody,RTail
Date,,,,,,,,,,,,,,,,,
2022-06-07 01:30:00,12555.75,12610.75,12534.25,12603.25,23152,12559.50,-3.75,43.75,0.35,47.50,76.50,0.61,7.50,21.50,9.80,62.09,28.10
2022-06-07 02:00:00,12603.50,12608.00,12555.75,12585.25,18868,12603.25,0.25,-18.00,-0.14,-18.25,52.25,0.42,4.50,29.50,8.61,34.93,56.46
2022-06-07 02:30:00,12580.50,12678.00,12580.50,12678.00,23336,12585.25,-4.75,92.75,0.74,97.50,97.50,0.78,0.00,0.00,0.00,100.00,0.00
2022-06-07 03:00:00,12679.50,12681.25,12585.50,12596.25,22824,12678.00,1.50,-81.75,-0.64,-83.25,95.75,0.76,1.75,10.75,1.83,86.95,11.23
2022-06-07 03:30:00,12589.75,12610.25,12558.50,12591.25,23554,12596.25,-6.50,-5.00,-0.04,1.50,51.75,0.41,19.00,31.25,36.71,2.90,60.39
2022-06-07 04:00:00,12589.25,12606.00,12542.00,12567.25,21935,12591.25,-2.00,-24.00,-0.19,-22.00,64.00,0.51,16.75,25.25,26.17,34.38,39.45
2022-06-07 04:30:00,12562.25,12615.75,12542.75,12606.75,35730,12567.25,-5.00,39.50,0.31,44.50,73.00,0.58,9.00,19.50,12.33,60.96,26.71
2022-06-07 05:00:00,12601.00,12601.25,12582.75,12591.75,10666,12606.75,-5.75,-15.00,-0.12,-9.25,18.50,0.15,0.25,9.00,1.35,50.00,48.65
2022-06-07 05:30:00,12591.25,12609.25,12591.25,12606.75,2090,12591.75,-0.50,15.00,0.12,15.50,18.00,0.14,2.50,0.00,13.89,86.11,0.00


In [21]:
df = candle('NQc1', '60', '2022-05-01 07:00:00', '2022-06-06')

In [22]:
df['t'] = df.index.strftime('%H:%M')

In [23]:
df.groupby('t')['Height_Rate'].describe()

,count,mean,std,min,25%,50%,75%,max
t,,,,,,,,
00:00,25.0,1.061600,0.436326,0.33,0.7300,1.090,1.2200,2.35
01:00,25.0,0.950400,0.363931,0.38,0.6800,0.900,1.1800,1.80
02:00,24.0,0.977083,0.464285,0.25,0.7375,0.835,1.1450,2.39
03:00,24.0,0.995000,0.576496,0.34,0.7250,0.915,1.1250,3.19
04:00,24.0,1.227917,0.578566,0.50,0.7700,1.080,1.5475,2.58
05:00,28.0,0.359286,0.250436,0.00,0.2375,0.320,0.4550,1.13
07:00,26.0,0.392308,0.186340,0.17,0.2725,0.345,0.4575,0.91
08:00,26.0,0.336923,0.181676,0.12,0.1950,0.270,0.4250,0.75
09:00,26.0,0.501154,0.272636,0.14,0.2900,0.475,0.6075,1.11


In [47]:
index_symbols_plot(['US30', 'US500', 'NQc1', 'RTYc1'], '2022-06-01','2022-06-06', '30')

In [33]:
price_symbol_plot('NQc1', '2022-05-01','2022-06-04', '5')

In [35]:
one_symbol_plot('NQc1', '2022-06-03', 5, '5')

In [5]:
df = fdr.DataReader('005930', '2022-01-01', '2022-06-04')
fdr.chart.plot(df, title='삼성전자(005930)')

Loading BokehJS ...

In [21]:
fdr.DataReader('UNRATE', data_source='fred').iplot()

In [64]:
xlsx = pd.read_excel('items.xlsx')


In [74]:
xlsx.set_index('symbol').T.to_dict()

{'GC': {'title': 'Gold',
  'type': 'Future',
  'exchange': 'COMEX',
  'code': 8830,
  'digit': 2},
 'SI': {'title': 'Silver',
  'type': 'Future',
  'exchange': 'COMEX',
  'code': 8836,
  'digit': 3},
 'HG': {'title': 'Copper',
  'type': 'Future',
  'exchange': 'COMEX',
  'code': 8831,
  'digit': 4},
 'CL': {'title': 'Crude Oil',
  'type': 'Future',
  'exchange': 'NYMEX',
  'code': 8849,
  'digit': 2},
 'NG': {'title': 'Natural Gas',
  'type': 'Future',
  'exchange': 'NYMEX',
  'code': 8862,
  'digit': 3}}